In [12]:
import numpy as np
import cv2 as cv
import statistics as st
import pytesseract
from PIL import ImageGrab
import time

MIN_AREA = 4
MIN_RATIO, MAX_RATIO = 0.2, 4
MIN_WIDTH, MIN_HEIGHT = 6, 12
MAX_WIDTH, MAX_HEIGHT = 256, 256
MAX_DIAG_MULTIPLYER = 20 # 5
MAX_ANGLE_DIFF = 15.0 # 12.0
MAX_AREA_DIFF = 0.5 # 0.5
MAX_WIDTH_DIFF = 1
MAX_HEIGHT_DIFF = 1
MIN_N_MATCHED = 3 # 3
PLATE_WIDTH_PADDING = 1.6 # 1.3
PLATE_HEIGHT_PADDING = 1.7 # 1.5
MIN_PLATE_RATIO = 3
MAX_PLATE_RATIO = 10

def process_img(original_image):
    #분석할 이미지를 불러온다.
    dst = original_image.copy()
    dst = original_image[200:600, 200:600]

    #이미지를 전처리한다. 크기변환과 그레이스케일 적용
    img_resize = cv.resize(dst, dsize=(800,800))
    img_gray = cv.cvtColor(img_resize, cv.COLOR_BGR2GRAY)


    #이미지 블러 -> 모폴로지 연산 -> 가우시안 임계값
    kernel = np.ones((3,3), np.uint8)
    img_blurred = cv.GaussianBlur(img_gray, ksize=(11,11), sigmaX=0)
    img_thresh = cv.adaptiveThreshold(img_blurred, maxValue=255.0, adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType=cv.THRESH_BINARY_INV,blockSize=19,C=9)
    img_morphed = cv.morphologyEx(img_thresh, cv.MORPH_OPEN,kernel)
    img_morphed = cv.morphologyEx(img_morphed, cv.MORPH_CLOSE,kernel)



    #흑백이미지에서 컨투어를 찾는다.
    contours, hierarchy = cv.findContours(img_morphed, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    height, width, channel = img_resize.shape
    temp_result = np.zeros((height, width, channel), np.uint8)

    for i in range(len(contours)):
        cv.drawContours(temp_result, [contours[i]], 0, (0,0,255), 2)

    contours_dict = []

    #찾은 컨투어를 포함하는 사각형 컨투어를 생성한다.
    for contour in contours:
        x,y,w,h = cv.boundingRect(contour)
        contours_dict.append({
        'contour' : contour,
        'x':x,
        'y':y,
        'w':w,
        'h':h,
        #중심점
        'cx' : x + (w/2),
        'cy' : y+ (h/2)
        })

    #사각형 컨투어에서 동/호수로 예상되는 사각형을 추려낸다
    possible_contours = []

    #크기가 평균보다 큰 컨투어를 추린다.
    #비율이 가로로 너무 길거나 세로로 너무 긴 컨투어는 제외한다.
    median_area = []

    for g in contours_dict:
        median_area.append(g['w']*g['h'])

    median = st.median(median_area)
    cnt = 0

    for d in contours_dict:
        area = d['w'] * d['h']
        ratio = d['w'] / d['h']
        if area > median \
            and MIN_RATIO < ratio < MAX_RATIO\
            and MIN_WIDTH < d['w'] < MAX_WIDTH\
            and MIN_HEIGHT < d['h'] < MAX_HEIGHT:
            d['idx'] = cnt
            cnt += 1
            possible_contours.append(d)

    #컨투어 리스트로부터 문자영역을 찾는 함수를 정의한다.
    #숫자간에 떨어진 거리와 각도를 이용한다.
    def find_chars(contour_list):
        matched_result_idx = []

        for d1 in contour_list:
            matched_contours_idx = []
            for d2 in contour_list:
                if d1['idx'] == d2['idx']:
                    continue

                dx = abs(d1['cx'] - d2['cx'])
                dy = abs(d1['cy'] - d2['cy'])

                diagonal_length1 = np.sqrt(d1['w'] ** 2 + d1['h'] ** 2)

                distance = np.linalg.norm(np.array([d1['cx'], d1['cy']]) - np.array([d2['cx'], d2['cy']]))
                if dx == 0:
                    angle_diff = 90
                else:
                    angle_diff = np.degrees(np.arctan(dy / dx))
                area_diff = abs(d1['w'] * d1['h'] - d2['w'] * d2['h']) / (d1['w'] * d1['h'])
                width_diff = abs(d1['w'] - d2['w']) / d1['w']
                height_diff = abs(d1['h'] - d2['h']) / d1['h']

                if distance < diagonal_length1 * MAX_DIAG_MULTIPLYER \
                and angle_diff < MAX_ANGLE_DIFF and area_diff < MAX_AREA_DIFF \
                and width_diff < MAX_WIDTH_DIFF and height_diff < MAX_HEIGHT_DIFF:
                    matched_contours_idx.append(d2['idx'])

            # append this contour
            matched_contours_idx.append(d1['idx'])

            if len(matched_contours_idx) < MIN_N_MATCHED:
                continue

            matched_result_idx.append(matched_contours_idx)

            unmatched_contour_idx = []
            for d4 in contour_list:
                if d4['idx'] not in matched_contours_idx:
                    unmatched_contour_idx.append(d4['idx'])

            unmatched_contour = np.take(possible_contours, unmatched_contour_idx)

            # recursive
            recursive_contour_list = find_chars(unmatched_contour)

            for idx in recursive_contour_list:
                matched_result_idx.append(idx)

            break

        return matched_result_idx


    #사각 컨투어를 이용해 문자영역을 찾는다.
    result_idx = find_chars(possible_contours)

    #문자영역을 결과 리스트에 저장한다.
    matched_result = []
    for idx_list in result_idx:
        matched_result.append(np.take(possible_contours, idx_list))

    temp_result = np.zeros((height,width,channel), dtype=np.uint8)


    #영역 이미지의 컨투어의 각도를 찾아서 이미지를 회전시킨다.
    plate_imgs = []
    plate_infos = []

    for i, matched_chars in enumerate(matched_result):
        sorted_chars = sorted(matched_chars, key=lambda x: x['cx'])

        plate_cx = (sorted_chars[0]['cx'] + sorted_chars[-1]['cx']) / 2
        plate_cy = (sorted_chars[0]['cy'] + sorted_chars[-1]['cy']) / 2

        plate_width = (sorted_chars[-1]['x'] + sorted_chars[-1]['w'] - sorted_chars[0]['x']) * PLATE_WIDTH_PADDING

        sum_height = 0
        for d in sorted_chars:
            sum_height += d['h']

        plate_height = int(sum_height / len(sorted_chars) * PLATE_HEIGHT_PADDING)

        triangle_height = sorted_chars[-1]['cy'] - sorted_chars[0]['cy']
        triangle_hypotenus = np.linalg.norm(
            np.array([sorted_chars[0]['cx'], sorted_chars[0]['cy']]) -
            np.array([sorted_chars[-1]['cx'], sorted_chars[-1]['cy']])
        )

        angle = np.degrees(np.arcsin(triangle_height / triangle_hypotenus))

        rotation_matrix = cv.getRotationMatrix2D(center=(plate_cx, plate_cy), angle=angle, scale=1.0)

        img_rotated = cv.warpAffine(img_morphed, M=rotation_matrix, dsize=(width, height))

        img_cropped = cv.getRectSubPix(
            img_rotated,
            patchSize=(int(plate_width), int(plate_height)),
            center=(int(plate_cx), int(plate_cy))
        )

        if img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO or img_cropped.shape[1] / img_cropped.shape[0] < MIN_PLATE_RATIO > MAX_PLATE_RATIO:
            continue

        plate_imgs.append(img_cropped)
        plate_infos.append({
            'x': int(plate_cx - plate_width / 2),
            'y': int(plate_cy - plate_height / 2),
            'w': int(plate_width),
            'h': int(plate_height)
        })

    for i, plate_img in enumerate(plate_imgs):
        plate_img = cv.resize(plate_img, dsize=(0, 0), fx=1.2, fy=1.2)
        _, plate_img = cv.threshold(plate_img, thresh=0.0, maxval=255.0, type=cv.THRESH_BINARY | cv.THRESH_OTSU)

        # find contours again (same as above)
        contours, _ = cv.findContours(plate_img, mode=cv.RETR_LIST, method=cv.CHAIN_APPROX_SIMPLE)

        plate_min_x, plate_min_y = plate_img.shape[1], plate_img.shape[0]
        plate_max_x, plate_max_y = 0, 0

        for contour in contours:
            x, y, w, h = cv.boundingRect(contour)

            area = w * h
            ratio = w / h

            if area > MIN_AREA \
            and w > MIN_WIDTH and h > MIN_HEIGHT \
            and MIN_RATIO < ratio < MAX_RATIO:
                if x < plate_min_x:
                    plate_min_x = x
                if y < plate_min_y:
                    plate_min_y = y
                if x + w > plate_max_x:
                    plate_max_x = x + w
                if y + h > plate_max_y:
                    plate_max_y = y + h

        img_result = plate_img[plate_min_y:plate_max_y, plate_min_x:plate_max_x]

        img_result = cv.GaussianBlur(img_result, ksize=(1, 1), sigmaX=0)
        _, img_result = cv.threshold(img_result, thresh=0.0, maxval=255.0, type=cv.THRESH_BINARY | cv.THRESH_OTSU)
        img_result = cv.copyMakeBorder(img_result, top=10, bottom=10, left=10, right=10, borderType=cv.BORDER_CONSTANT, value=(0,0,0))

        # 이미지로부터 동호수를 검출한다
        chars = pytesseract.image_to_string(img_result,lang='eng', config='--psm 6  --oem 1 -c tessedit_char_whitelist=-_0123456789abcdefghijklmnpqrstuvwxyz')
        chars = chars.rstrip()
        chars = chars.lstrip()
        chars = chars.replace('?','')

    # 처리된 이미지와 인식된 동/호수를 출력으로 리턴한다.
    try:
        return img_morphed, img_result, chars

    # 재생빈도의 차이로 발생하는 에러에 대한 예외처리
    except UnboundLocalError:
        time.sleep(0.02)



while(True):
    # (0,40)부터 (800,600)애 해당하는 영상을 screen 변수에 담는다.
    screen = np.array(ImageGrab.grab(bbox=(0,40,800,840)))
    text_found = ''
    try:
    # 이미지 처리가된 영상과 해당 영상에서 인식한 동/호수를 변수에 담는다.
        processed_screen, new_screen, text_found = process_img(screen)
        second_screen = np.zeros((200,400,3),np.uint8)
    #pyapt이라는 이름의 창을 생성하고 이 창에 screen 이미지를 뿌려줍니다
        cv.putText(second_screen, text_found, (10,100), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255))
        cv.imshow('pyapt_process', processed_screen)
        cv.imshow('pyapt_image', new_screen)
        cv.imshow('pyapt_number', second_screen)
    except :
        time.sleep(0.03)



    # 'q'키를 누르면 종료합니다
    if cv.waitKey(25) & 0xFF == ord('q'):
        cv.destroyAllWindows()
        break